In [231]:
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [232]:
data = pd.read_csv("../Data/vix/TRBContest_Deliverables/trip_leg_matrix_2014_04_27_DDP.csv")
print data.shape

(131770, 9)


In [233]:
print data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131770 entries, 0 to 131769
Data columns (total 9 columns):
Origin_Zone         131770 non-null int64
Destination_Zone    131770 non-null int64
Start_Date          131770 non-null int64
End_Date            131770 non-null int64
Aggregation         131770 non-null object
Subscriber_Class    131770 non-null object
Purpose             131770 non-null object
Time_of_Day         131770 non-null object
Count               131770 non-null float64
dtypes: float64(1), int64(4), object(4)
memory usage: 9.0+ MB
None


In [234]:
print data.describe()

        Origin_Zone  Destination_Zone  Start_Date    End_Date          Count
count  1.317700e+05      1.317700e+05    131770.0    131770.0  131770.000000
mean   1.176292e+14      1.238370e+14  20140427.0  20140427.0      80.548865
std    3.221698e+14      3.293967e+14         0.0         0.0     575.678409
min    1.000000e+00      1.000000e+00  20140427.0  20140427.0       3.000000
25%    5.130000e+02      5.140000e+02  20140427.0  20140427.0      28.450000
50%    8.970000e+02      8.970000e+02  20140427.0  20140427.0      39.290000
75%    1.106000e+03      1.109000e+03  20140427.0  20140427.0      52.660000
max    1.000000e+15      1.000000e+15  20140427.0  20140427.0   27877.540000


In [235]:
ordes = data[["Origin_Zone","Destination_Zone"]]

ordes.ix[ordes.Origin_Zone > 2000] %= 5000 
ordes.ix[ordes.Destination_Zone > 2000] %= 5000
print ordes.head()

   Origin_Zone  Destination_Zone
0          897              4005
1         4000               310
2         1235              1318
3          799              4005
4         4001              4001


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [236]:
time_division = data[["Time_of_Day"]]
#print time_division.head()

def start_cleaner(value):
    """
    Clean and divide into start and end time
    """
    def start_time(val):
        return int(val[1:3])
    
    start = value.apply(start_time)
    return start

def end_cleaner(value):
    def end_time(val):
        return int(val[5:]) 
    end = value.apply(end_time)
    return end
        
x = time_division.apply(start_cleaner)
y = time_division.apply(end_cleaner)
z = pd.concat([x,y],axis = 1)
z.columns = ["Start_time","End_time"]
print z.head()

   Start_time  End_time
0           0         1
1           0         1
2           2         3
3           1         2
4           0         1


In [237]:
date = data.Start_Date.apply(lambda x : x%100)
print date.head()

0    27
1    27
2    27
3    27
4    27
Name: Start_Date, dtype: int64


In [238]:
data = data[["Aggregation","Subscriber_Class","Purpose","Count"]]
print data.head()

  Aggregation    Subscriber_Class Purpose   Count
0         Sun  Short Term Visitor      OH  114.49
1         Sun         Home Worker      OH   58.72
2         Sun         Home Worker      OH   22.08
3         Sun  Short Term Visitor      OH  413.94
4         Sun   Long Term Visitor      OO  440.72


In [239]:
final_data = pd.concat([ordes,date,data,z],axis = 1)
print final_data.head()

   Origin_Zone  Destination_Zone  Start_Date Aggregation    Subscriber_Class  \
0          897              4005          27         Sun  Short Term Visitor   
1         4000               310          27         Sun         Home Worker   
2         1235              1318          27         Sun         Home Worker   
3          799              4005          27         Sun  Short Term Visitor   
4         4001              4001          27         Sun   Long Term Visitor   

  Purpose   Count  Start_time  End_time  
0      OH  114.49           0         1  
1      OH   58.72           0         1  
2      OH   22.08           2         3  
3      OH  413.94           1         2  
4      OO  440.72           0         1  


In [240]:
final_data.to_csv("matrix_27.csv",index = False)